## Part 1: Preprocessing

In [456]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

from sklearn.preprocessing import LabelEncoder, OneHotEncoder


#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [457]:
# Determine the number of unique values in each column
display(attrition_df.nunique())

# [Optional] Attition and Department value counts:
# display(attrition_df["Attrition"].value_counts())
# display(attrition_df["Department"].value_counts())
# [Optional] List columns
# display(attrition_df.columns)

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [458]:
# Create y_df (as a copy of attrition_df) with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]].copy()

In [459]:
# Create a list of at least 10 column names to use as X data
# NOTE: I am using 12 columns
column_names = [x_column for x_column in attrition_df.columns if x_column not in [
    "Attrition", "Department", "DistanceFromHome","Education",  
    "MaritalStatus", "PercentSalaryHike", "EducationField", "JobRole",
    "PerformanceRating", "RelationshipSatisfaction", "StockOptionLevel", 
    "TotalWorkingYears", "TrainingTimesLastYear", "YearsInCurrentRole", 
    "YearsWithCurrManager", "YearsAtCompany", "YearsSinceLastPromotion"]]

# Create X_df as copy of attrition_df, using your selected columns
X_df = attrition_df[column_names].copy()
display(X_df)

# Show the data types for X_df
display(X_df.dtypes)

,Age,BusinessTravel,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,OverTime,WorkLifeBalance
0,41,Travel_Rarely,2,94,3,2,4,8,Yes,1
1,49,Travel_Frequently,3,61,2,2,2,1,No,3
2,37,Travel_Rarely,4,92,2,1,3,6,Yes,3
3,33,Travel_Frequently,4,56,3,1,3,1,Yes,3
4,27,Travel_Rarely,1,40,3,1,2,9,No,3
...,...,...,...,...,...,...,...,...,...,...
1465,36,Travel_Frequently,3,41,4,2,4,4,No,3
1466,39,Travel_Rarely,4,42,2,3,1,4,No,3
1467,27,Travel_Rarely,2,87,4,2,2,1,Yes,3
1468,49,Travel_Frequently,4,63,2,2,2,2,No,2


Age                         int64
BusinessTravel             object
EnvironmentSatisfaction     int64
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
JobSatisfaction             int64
NumCompaniesWorked          int64
OverTime                   object
WorkLifeBalance             int64
dtype: object

# Perform Label Encoding For binary labels Feature Encoding on Nominal 
# Fields since it is binary no risk of false ordinal relationships
# X_df -> OverTime
# y_df -> Attrition

In [460]:

# [Optional] Encode X_df and y_df fields
print("🚀 Checking columns in X_df before splitting:")
print(X_df.columns)

# Step 1: Label Encode `OverTime` (Yes=1, No=0)
# Initialize LabelEncoder
overtime_le = LabelEncoder()
display(X_df["OverTime"])
# Perform Fit Transform
X_df["OverTime"] = overtime_le.fit_transform(X_df["OverTime"])

# Step 2: LabelEncoder 'Attrition'  (Yes=1, No=0)
attrition_le = LabelEncoder()

# Perform Fit Transform
y_df["Attrition"] = attrition_le.fit_transform(y_df[["Attrition"]])

# Step 3: Check if y_df['Attrition'] and X_df['OverTime'] is encoded
display(X_df)
display(y_df)



🚀 Checking columns in X_df before splitting:
Index(['Age', 'BusinessTravel', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked',
       'OverTime', 'WorkLifeBalance'],
      dtype='object')


0       Yes
1        No
2       Yes
3       Yes
4        No
       ... 
1465     No
1466     No
1467    Yes
1468     No
1469     No
Name: OverTime, Length: 1470, dtype: object

/Users/tlockhart/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Age,BusinessTravel,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,OverTime,WorkLifeBalance
0,41,Travel_Rarely,2,94,3,2,4,8,1,1
1,49,Travel_Frequently,3,61,2,2,2,1,0,3
2,37,Travel_Rarely,4,92,2,1,3,6,1,3
3,33,Travel_Frequently,4,56,3,1,3,1,1,3
4,27,Travel_Rarely,1,40,3,1,2,9,0,3
...,...,...,...,...,...,...,...,...,...,...
1465,36,Travel_Frequently,3,41,4,2,4,4,0,3
1466,39,Travel_Rarely,4,42,2,3,1,4,0,3
1467,27,Travel_Rarely,2,87,4,2,2,1,1,3
1468,49,Travel_Frequently,4,63,2,2,2,2,0,2


,Attrition,Department
0,1,Sales
1,0,Research & Development
2,1,Research & Development
3,0,Research & Development
4,0,Research & Development
...,...,...
1465,0,Research & Development
1466,0,Research & Development
1467,0,Research & Development
1468,0,Sales


In [461]:
# [Optional] Determine how the X_df['Business'] columns need to be split
display(attrition_df["BusinessTravel"].value_counts())

BusinessTravel
Travel_Rarely        1043
Travel_Frequently     277
Non-Travel            150
Name: count, dtype: int64

# Perform Test Train Split

In [462]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# Train-Test Split (Before OneHotEncoding!)
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

print("🚀 Checking columns in X_train after splitting:")
print(X_train.columns)



🚀 Checking columns in X_train after splitting:
Index(['Age', 'BusinessTravel', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked',
       'OverTime', 'WorkLifeBalance'],
      dtype='object')


# Perform OneHotEncode 
# Nominal X_df -> BusinessTravel 

In [463]:
# Convert your X data to numeric data types however you see fit
# Step 1. One-Hot Encode `BusinessTravel`
business_ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

# Fit on training data only and transform both train & test sets
X_train_business_encoded = business_ohe.fit_transform(X_train[["BusinessTravel"]])
X_test_business_encoded = business_ohe.transform(X_test[["BusinessTravel"]])

# Convert to DataFrame with column names
business_columns = business_ohe.get_feature_names_out(["BusinessTravel"])
X_train_business_df = pd.DataFrame(X_train_business_encoded, columns=business_columns, index=X_train.index)
X_test_business_df = pd.DataFrame(X_test_business_encoded, columns=business_columns, index=X_test.index)


# Concatenate with original X_train and X_test
X_train = pd.concat([X_train, X_train_business_df], axis=1)
X_test = pd.concat([X_test, X_test_business_df], axis=1)

# Drop Original Categorical Data:
X_train = X_train.drop(columns=["BusinessTravel"])
X_test = X_test.drop(columns=["BusinessTravel"])

# [Optional] Step 2: Check if X_train and X_test BusinessTravels are encoded
display(X_train.head())
display(X_train.head())

# Check X_train value counts
display(X_train['OverTime'].value_counts)

,Age,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,OverTime,WorkLifeBalance,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely
1097,24,3,57,2,1,1,0,0,3,0.0,0.0,1.0
727,18,2,73,3,1,4,1,0,3,1.0,0.0,0.0
254,29,4,45,3,2,4,2,0,3,0.0,0.0,1.0
1175,39,4,66,3,2,2,4,0,3,0.0,0.0,1.0
1341,31,2,89,3,2,3,1,0,3,0.0,0.0,1.0


,Age,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,NumCompaniesWorked,OverTime,WorkLifeBalance,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely
1097,24,3,57,2,1,1,0,0,3,0.0,0.0,1.0
727,18,2,73,3,1,4,1,0,3,1.0,0.0,0.0
254,29,4,45,3,2,4,2,0,3,0.0,0.0,1.0
1175,39,4,66,3,2,2,4,0,3,0.0,0.0,1.0
1341,31,2,89,3,2,3,1,0,3,0.0,0.0,1.0


<bound method IndexOpsMixin.value_counts of 1097    0
727     0
254     0
1175    0
1341    0
       ..
1130    0
1294    0
860     1
1459    1
1126    1
Name: OverTime, Length: 1176, dtype: int64>

In [464]:
# [Optiona] Make sure X_test and X_train variables are numeric
print(X_test.dtypes)
print()
print(X_train.dtypes)

Age                                   int64
EnvironmentSatisfaction               int64
HourlyRate                            int64
JobInvolvement                        int64
JobLevel                              int64
JobSatisfaction                       int64
NumCompaniesWorked                    int64
OverTime                              int64
WorkLifeBalance                       int64
BusinessTravel_Non-Travel           float64
BusinessTravel_Travel_Frequently    float64
BusinessTravel_Travel_Rarely        float64
dtype: object

Age                                   int64
EnvironmentSatisfaction               int64
HourlyRate                            int64
JobInvolvement                        int64
JobLevel                              int64
JobSatisfaction                       int64
NumCompaniesWorked                    int64
OverTime                              int64
WorkLifeBalance                       int64
BusinessTravel_Non-Travel           float64
BusinessTravel_Tr

In [465]:
# Create a StandardScaler
sc = StandardScaler()

# Scale the training and testing data
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [466]:
from sklearn.preprocessing import OneHotEncoder


display(X_train_scaled)
display(X_test_scaled)


array([[-1.38855944,  0.2797056 , -0.47283217, ..., -0.32604128,
        -0.49041445,  0.63904869],
       [-2.04073779, -0.63910385,  0.30937375, ...,  3.06709629,
        -0.49041445, -1.56482598],
       [-0.84507748,  1.19851504, -1.05948661, ..., -0.32604128,
        -0.49041445,  0.63904869],
       ...,
       [-1.60595222,  0.2797056 , -0.912823  , ..., -0.32604128,
         2.03909165, -1.56482598],
       [-0.84507748,  1.19851504, -1.01059874, ..., -0.32604128,
        -0.49041445,  0.63904869],
       [ 1.43754676,  0.2797056 , -0.37505643, ..., -0.32604128,
        -0.49041445,  0.63904869]])

array([[-0.95377387,  1.19851504,  0.84714032, ..., -0.32604128,
        -0.49041445,  0.63904869],
       [ 1.76363593,  1.19851504, -0.47283217, ..., -0.32604128,
        -0.49041445,  0.63904869],
       [-1.38855944,  1.19851504, -0.4239443 , ..., -0.32604128,
        -0.49041445,  0.63904869],
       ...,
       [ 1.43754676,  1.19851504, -1.15726235, ..., -0.32604128,
        -0.49041445,  0.63904869],
       [ 0.02449366,  1.19851504,  0.65158884, ..., -0.32604128,
        -0.49041445,  0.63904869],
       [ 1.11145758,  0.2797056 , -1.54836531, ..., -0.32604128,
        -0.49041445,  0.63904869]])

# Perform OneHotEncoding on 
# y_df['Department']

In [467]:
# Step 1: Create a OneHotEncoder for the Department column
department_ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore" )

# Convert your y data to numeric data types however you see fit
# Fit the encoder to the training data
y_train_department_encoded = department_ohe.fit_transform(y_train[["Department"]])
y_test_department_encoded = department_ohe.transform(y_test[["Department"]])

# Convert departments to dataFrame with column names
department_columns = department_ohe.get_feature_names_out(["Department"])

# Create two new variables by applying the encoder
# to the numpy_array converted training and testing data
y_train_department_df = pd.DataFrame(y_train_department_encoded, columns=department_columns, index=y_train.index)
y_test_department_df = pd.DataFrame(y_test_department_encoded, columns=department_columns, index=y_test.index)

# Concatenate with original y_train and y_test
y_train = pd.concat([y_train, y_train_department_df], axis=1)
y_test = pd.concat([y_test, y_test_department_df], axis=1)

# Drop Original Categorical Data:
y_train = y_train.drop(columns=["Department"])
y_test = y_test.drop(columns=["Department"])

# Step 2: Check if y_train and y_test Department are Encoded
display(y_train.head())
display(y_test.head())


,Attrition,Department_Human Resources,Department_Research & Development,Department_Sales
1097,0,0.0,1.0,0.0
727,0,0.0,1.0,0.0
254,0,0.0,0.0,1.0
1175,0,0.0,1.0,0.0
1341,0,0.0,1.0,0.0


,Attrition,Department_Human Resources,Department_Research & Development,Department_Sales
1041,0,0.0,0.0,1.0
184,0,0.0,1.0,0.0
1222,1,1.0,0.0,0.0
67,0,0.0,1.0,0.0
220,0,0.0,1.0,0.0


## Part 2: Create, Compile, and Train the Model

In [468]:
# Find the number of columns in the X training data.
X_train_columns = X_train_scaled.shape[1]
display(X_train_columns)


# Create the input layer with the number of columns
input_layer = layers.Input(shape=(X_train_scaled.shape[1],), name="input_features")

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation="relu")(input_layer)
shared_backbone_layer = layers.Dense(128, activation="relu")(shared_layer1)

12

In [469]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_dense = layers.Dense(32, activation="relu")(shared_backbone_layer)

# Create the output layer (Departments are MultiClass and Mutually Exclusive -> Sigmoid)
# Q1: WHY DO WE USE SOFTMAX ACTIVATION FUNCTION
# Note: y_train_department_df contains the one-hot encoded department classes
# IMPORTANT: THE NAME OF YOU LAYER MUST MATCH THE NAME IN MODEL.fit
department_output = layers.Dense(len(y_train_department_df.columns),
                                 activation='softmax',
                                 name='department_output')(department_dense)

In [470]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_dense = layers.Dense(32, activation="relu")(shared_backbone_layer)

# Create the output layer
# NOTE: Since Binary Attrition is a Bnary Classification just use 1 instead of
# len(y_train['Attrition'].columns
# IMPORTANT: THE NAME OF YOU LAYER MUST MATCH THE NAME IN MODEL.fit
attrition_output = layers.Dense(1,
                                 activation='sigmoid',
                                 name='attrition_output')(attrition_dense)

In [ ]:
# Create the model
model = Model(inputs=input_layer, outputs={
    'department_output':department_output,
    'attrition_output': attrition_output
})

# Compile the model
# Q2: Since I am using 12 columns input_layer in summary is accurate
# Q3: Since I used labelEncoder to Encode the Attrition label, 
# we only need 1 neuron to represent the binary output
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': ['accuracy'],
                       'attrition_output': ['accuracy']},
              run_eagerly=True)

# Summarize the model
model.summary()

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 12)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_59 (Dense)    │ (None, 64)        │        832 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_60 (Dense)    │ (None, 128)       │      8,320 │ dense_59[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_62 (Dense)    │ (None, 32)        │      4,128 │ dense_60[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_61 (Dense)    │ (None, 32)        │      4,128 │ dense_60[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │         33 │ dense_62[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_61[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,540 (68.52 KB)

 Trainable params: 17,540 (68.52 KB)

 Non-trainable params: 0 (0.00 B)

In [472]:
# Train the model

# 🚀 Step 1: Ensure X_train_scaled is a NumPy array
X_train_scaled_np = np.array(X_train_scaled)

# 🚀 Step 2: Ensure y_train is formatted correctly
y_train_department_np = np.array(y_train_department_df) # One-hot encoded labels

# IMPORTANT: Make Sure the Columns is a 2D array no 1D
y_train_attrition_np = np.array(y_train["Attrition"]).reshape(-1, 1)

y_train_dict = {
    "department_output": y_train_department_np,
    "attrition_output": y_train_attrition_np  
}

model.fit(
    X_train_scaled_np,
    y_train_dict,
    epochs=10,  # You can adjust the number of epochs based on your needs
    batch_size=32,  # You can adjust the batch size based on your available memory
    validation_split=0.2  # You can specify the validation split if you have a separate validation set
)

Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - attrition_output_accuracy: 0.7894 - attrition_output_loss: 0.5751 - department_output_accuracy: 0.5170 - department_output_loss: 0.9949 - loss: 1.5699 - val_attrition_output_accuracy: 0.7966 - val_attrition_output_loss: 0.5169 - val_department_output_accuracy: 0.6314 - val_department_output_loss: 0.7906 - val_loss: 1.3075
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - attrition_output_accuracy: 0.8458 - attrition_output_loss: 0.4138 - department_output_accuracy: 0.6722 - department_output_loss: 0.7449 - loss: 1.1586 - val_attrition_output_accuracy: 0.7966 - val_attrition_output_loss: 0.4447 - val_department_output_accuracy: 0.6356 - val_department_output_loss: 0.7848 - val_loss: 1.2295
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - attrition_output_accuracy: 0.8469 - attrition_output_loss: 0.3816 - department_output_accuracy: 0.6803 - department_output_loss: 0.7080 - loss: 1.0896 - val_attrition_output_accuracy: 0.7966 - 

In [473]:
# Evaluate the model with the testing data

# Step1: Split y_test into separate NumPy arrays for  
# each output and pass them as a dictionary
X_test_np = np.array(X_test_scaled)
y_test_department_np = np.array(y_test_department_df)

# IMPORTANT: Make Sure the Columns is a 2D array no 1D
y_test_attrition_np = np.array(y_test["Attrition"]).reshape(-1, 1)

# Step2: Combine y_test components into a dictionary:
y_test_dict = {
    'department_output': y_test_department_np, # Multi-class labels
    'attrition_output': y_test_attrition_np # Binary labels
}
results = model.evaluate(X_test_np, y_test_dict)
# 🚀 Print available metric names
print("Available Metrics:", model.metrics_names)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - attrition_output_accuracy: 0.8543 - attrition_output_loss: 0.3771 - department_output_accuracy: 0.6189 - department_output_loss: 0.8292 - loss: 1.2063
Available Metrics: ['loss', 'compile_metrics', 'department_output_loss', 'attrition_output_loss']


In [ ]:
# Q4: Why doesn't my metics contain in accuracy information?

# Print the accuracy for both department and attrition
# 🚀 Extract accuracy values
department_accuracy = results[model.metrics_names.index("department_output_accuracy")]
attrition_accuracy = results[model.metrics_names.index("attrition_output_accuracy")]

# 🚀 Print the accuracy scores
print(f"🎯 Department Classification Accuracy: {department_accuracy:.4f}")
print(f"🎯 Attrition Classification Accuracy: {attrition_accuracy:.4f}")

ValueError: 'department_output_accuracy' is not in list

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 